In [142]:
import numpy as np
import pandas as pd
import glob, os
import itertools
import matplotlib.image as mpimg
import pickle
import math
import json

# import classifier

%matplotlib inline
import matplotlib.pyplot as plt

In [169]:
# TSD dataset
failure_starts = pd.read_csv('/Users/kenneth/Projects/ga-ml/tsd/alerted.csv', index_col=0)
failure_starts['start'] = -1
failure_starts.update(pd.read_csv('/Users/kenneth/Projects/ga-ml/tsd/failure_starts.csv', index_col=0))
failure_starts['start'] = failure_starts['start']*25/30 # account for the 25fps vs 30 fps bug in the code.

def load_df_from_json(fn):
    if not os.path.isfile(fn):
        return pd.DataFrame()
    l = [ f['fields'] for f in json.load(open(fn, 'r')) if 'fields' in f]
    df = pd.DataFrame(l)
    return df

# def validate_ewm(fn):
#     df = load_df_from_json(fn)
#     if df.empty:
#         return
#     ser = ((df['ewm_mean'] - df['rolling_ewm'])[20:] > 1e-03)
#     return ser[ser].count() == 0

# for vid in alerted.index.values:
#     print(vid)
#     print(validate_ewm('/mnt/ml-data4/tsd/jsons/{}.json'.format(vid)))

def load_detections(vid):
    return load_df_from_json('/Users/kenneth/Projects/ga-ml/tsd/jsons/{}.json'.format(vid))

vids = [str(i) for i in failure_starts.index.values.tolist()]

In [170]:
settings = dict(THRESHOLD_LOW=0.11, THRESHOLD_HIGH=0.65, INIT_SAFE_FRAME_NUM=30, ROLLING_MEAN_SHORT_MULTIPLE=5.43)

def is_failing(prediction, detective_sensitivity=1.0):
    if prediction.current_frame_num < settings['INIT_SAFE_FRAME_NUM']:
        return False

    adjusted_ewm_mean = prediction.ewm_mean * detective_sensitivity
    if adjusted_ewm_mean < settings['THRESHOLD_LOW'] + prediction.rolling_mean_long:
        return False

    if adjusted_ewm_mean > settings['THRESHOLD_HIGH'] + prediction.rolling_mean_long:
        return True

    if adjusted_ewm_mean > prediction.rolling_mean_short * settings['ROLLING_MEAN_SHORT_MULTIPLE']:
        return True
    
    return False

In [171]:
def video_fp_fn(vids, plot=False):
    if plot:
        plt.style.use('ggplot')
        fig=plt.figure(figsize=(16, 16*math.ceil(len(vids)/7)))

    fp = []
    fn = []
    tn = []
    tp = []
    for j, vid in enumerate(vids):
        failure_start = failure_starts.loc[int(vid)].start
        det = load_detections(vid)
        det['is_failing'] = dets.apply(is_failing, axis=1)
        failing_frames = dets[dets['is_failing']].index

        if len(failing_frames) > 0:
            detected_failure_start = dets[dets['is_failing']].index[0]
            if failure_start < 0:
                fp += [vid]
            else:
                if detected_failure_start < failure_start:
                    fp += [vid]
                else:
                    tp += [vid]
        else:
            if failure_start < 0:
                tn += [vid]
            else:
                fn += [vid]
    return fp, fn, tn, tp

# dets = load_detections(143)
# dets['is_failing'] = dets.apply(is_failing, axis=1)
# video_fp_fn([143])
# dets[dets.is_failing]
(fp, fn, tn, tp) = video_fp_fn(vids)

In [172]:
print(len(tp))

127
